# Datová akademie, ČSOB, 2023

---

* [Spojování dat](#Spojování-dat),
    - [spojování s concat](#Spojování-s-concat),
    - [spojování s concat a arg. join](#concat-a-volitelný-argument-join),
    - [spojování s metodou append](#Spojování-pomocí-metody-APPEND),
    - [spojování s merge](#Spojování-pomocí-merge),
    - [spojování s metodou join](#Spojení-pomocí-indexů,-join),
    - [cvičení 1]().
* [Agregace](#Agregace),
    - [jednoduchá agregace](#Jednoduchá-agregace),
    - [seskupování groupby](#Seskupování-groupby),
    - [agregace](#Agregace-s-metodami),
    - [filtrování](#Filtrování),
    - [transformace](#Transformace),
    - [apply](#Metoda-apply),
    - [cvičení 2]().
* [Pivot tabulka](#Pivot-tabulky),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti](),
    - [cvičení 3]().

<br>

## Spojování dat

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.APOjuHGvkM0wQaUx9ELKAgHaHa%26pid%3DApi&f=1&ipt=a8b2b692c40e190d4b28c164bf13e6c944702a69944035a4ef9053aa6ea1b190&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů** až po složitější joiny podobně jako u databází.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce.

* funkce `concat`,
* funkce `merge`,
* metoda `append` (outdated),
* metoda `join`.

<br>

### Spojování s CONCAT

---

Funkce `concat()` v `pandas` umožňuje spojit **dva nebo více**:
1. sloupečků,
2. tabulek.

<br>

*Spojení* můžeš provést:
1. **horizontálně**, po sloupcích,
2. **vertikálně**, po řádcích.

Použití funkce `concat()` spočívá **v předání seznamu objektů**, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků

In [ ]:
from pandas import concat, Series, DataFrame

In [ ]:
sloupec_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupec_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [ ]:
sloupec_1

In [ ]:
sloupec_2

In [ ]:
spojene_sloupce = concat([sloupec_1, sloupec_2])

In [ ]:
spojene_sloupce = concat([sloupec_1, sloupec_2], axis=1)

In [ ]:
spojene_sloupce = concat([sloupec_1, sloupec_2], axis=0)

In [ ]:
spojene_sloupce

#### Spojení tabulek

In [ ]:
uzivatele_1 = {
    'jmeno': ['Matouš', 'Marek', 'Lukáš'],
    'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Petr', 'Jan', 'Michal'],
    'vek': [40, 45, 50]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
df_spojene = concat([df_uzivatele_1, df_uzivatele_2], axis=0)

In [ ]:
df_spojene.head(6)

Tady se ovšem **zduplikovali hodnoty** některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [ ]:
df_spojene = df_spojene.reset_index()

In [ ]:
df_spojene.head(6)

In [ ]:
df_spojene = df_spojene.drop("index", axis=1)  # pop, del

In [ ]:
df_spojene.index

In [ ]:
df_spojene.columns

In [ ]:
df_spojene

In [ ]:
df_spojene.index.names

<br>

Doplníš jméno pro Index:

In [ ]:
df_spojene.index.names = "Muj_Index"

In [ ]:
df_spojene.index.names = ["Muj_Index"]

In [ ]:
df_spojene.index.names

In [ ]:
df_spojene

Pro odchytávání duplicitních indexů můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

### CONCAT a volitelný argument JOIN

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo **tabulky a sloupečky spojit**.

To prakticky není vždy ideální řešení, protože některé sloupečky můžou, ale nemusí být shodné.

In [ ]:
from pandas import DataFrame, concat

In [ ]:
df_vzorek_1 = DataFrame(
    {"A": ["A1", "A2"], "B": ["B1", "B2"], "C": ["C1", "C2"]},
    index=[1, 2]
)

In [ ]:
df_vzorek_2 = DataFrame(
    {"B": ["B3", "B4"], "C": ["C3", "C4"], "D": ["D3", "D4"]},
    index=[3, 4]
)

In [ ]:
df_vzorek_1

In [ ]:
df_vzorek_2

In [ ]:
vystup = concat([df_vzorek_1, df_vzorek_2])  # axis=0

In [ ]:
vystup = concat([df_vzorek_1, df_vzorek_2], axis=0)  # axis=0

In [ ]:
vystup

Pokud některá data chybějí, jsou automaticky vyplněná **neznámými hodnotami**.

Řešením takové situace můžeš být zavedení jiného způsobu spojování.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [ ]:
vystup_bez_na = concat([df_vzorek_1, df_vzorek_2], join='inner', axis=0)

In [ ]:
vystup_bez_na

### Spojování pomocí metody APPEND

---

Jde **o zastaralé řešení**, ale přesto se s ním můžeš setkat.

Jelikož je spojování natolik běžnou operací, vznikl ještě jeden způsob, který je dokonce stručnější jako `concat`.

Jde o metodu `append`:

In [ ]:
df_vzorek_3 = DataFrame({"A": ["A1", "A2"], "B": ["B1", "B2"]}, index=[1, 2])

In [ ]:
df_vzorek_4 = DataFrame({"A": ["A3", "A4"], "B": ["B3", "B4"]}, index=[3, 4])

In [ ]:
df_vzorek_3

In [ ]:
df_vzorek_4

In [ ]:
df_vzorek_3.append(df_vzorek_4)  # 1.5.3

Tato metoda přitom neupravovala původní objekty (jako `append` a `extend` pro `list`), ale **vytvořila nový objekt**.

### Spojování pomocí `merge`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní **definovat sloupec, nebo sloupce**, na kterých chceš spojení provést (parametr `on`).

In [ ]:
from pandas import merge

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
     'vek': [25, 30, 35, 40],
     'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
     'pocet_prijemcu': [100, 200, 150, 250]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vystup = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='outer')

In [ ]:
vystup = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='inner')  # how='inner'

In [ ]:
vystup

Obecně platí, že pokud potřebuješ *spojit* dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vystup_left_join = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='left')

In [ ]:
vystup_left_join

V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš celou první (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [ ]:
df8 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [1, 2, 3, 4]
})

In [ ]:
df9 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [3, 1, 4, 2]
})

In [ ]:
df8

In [ ]:
df9

In [ ]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [ ]:
vystup_konflikt

Funkce sama **doplní přípony**, aby rozlišila **mezi oběma původními sloupci**.

**Pokud potřebuješ vlastní přípony**, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [ ]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_DF8', '_DF9'))

In [ ]:
vystup_vlastni_pripony

### Spojení pomocí indexů, `join`

---

Tato metoda slouží k propojení dvou DataFrame objektů na základě **jejich indexů nebo hodnot**.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
     'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
    'pocet_prijemcu': [100, 150, 200]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
df_uzivatele_2.index.names

In [ ]:
df_uzivatele_1 = df_uzivatele_1.set_index('jmeno')

In [ ]:
df_uzivatele_2 = df_uzivatele_2.set_index('jmeno')

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vysledek_join_metody = df_uzivatele_1.join(df_uzivatele_2)

In [ ]:
vysledek_join_metody

Hlavní rozdíl mezi těmito funkcemi je způsob určení **sloupce nebo sloupců**, podle kterých se má propojení provést.

1. Metoda `join()` propojuje DataFrame objekty **na základě jejich indexů**.
2. Funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij spojovací funkci pro spojení objektů typu `DataFrame` , které vytvoříš z proměnných `data_zamestnanci_1` a `data_zamestnanci_2`,
2. použij spojovací funkci, které zadáš vzniklý objekt typu `DataFrame`  z bodu 1. a druhý `DataFrame`, který vytvoříš z proměnné `data_oddeleni` (spoj přes sloupeček `"oddělení_id"`),
3. vytvoř výsledný objekt typu `DataFrame` , který obsahuje pouze zaměstnance z oddělení `IT`.

In [ ]:
data_zamestnanci_1 = {
    'id': [1, 2, 3],
    'jméno': ['Jan', 'Marie', 'Petr'],
    'příjmení': ['Novák', 'Svobodová', 'Dvořák'],
    'oddělení_id': [100, 200, 100]
}

data_zamestnanci_2 = {
    'id': [4, 5],
    'jméno': ['Anna', 'Josef'],
    'příjmení': ['Kopecká', 'Vondráček'],
    'oddělení_id': [200, 100]
}

data_oddeleni = {
    'oddělení_id': [100, 200],
    'název_oddělení': ['IT', 'HR']
}

In [ ]:
df1, df2 = ...

In [ ]:
df_spojeni_zam = pd.concat([df1, df2], axis=0, ignore_index=True)

In [ ]:
df1 = pd.concat(
    [pd.DataFrame(data) for data in [data_zamestnanci_1, data_zamestnanci_2]]
).reset_index(drop = True).set_index('id')

In [ ]:
df_spojeni_odd = pd.merge(df_oddeleni, on='oddělení_id', how=)

In [ ]:
df2 = df1.merge( pd.DataFrame( data_oddeleni ), on = 'oddělení_id', how = 'left' )

In [ ]:
df3 = df2[df2['název_oddělení'] == 'IT']

In [ ]:
display( df1, df2, df3 )

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    df_zamestnanci_1 = pd.DataFrame(data_zamestnanci_1)
    df_zamestnanci_2 = pd.DataFrame(data_zamestnanci_2)
    df_oddeleni = pd.DataFrame(data_oddeleni)

    df_vsichni_zamestnanci = pd.concat([df_zamestnanci_1, df_zamestnanci_2], ignore_index=True)

    df_vsechna_oddeleni = pd.merge(df_vsichni_zamestnanci, df_oddeleni, on="oddělení_id")

    df_jen_it = df_vsechna_oddeleni[df_vsechna_oddeleni["název_oddělení"] == "IT"]
    ```
</details>

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [ ]:
from pandas import read_csv

In [ ]:
df_nemovitosti = read_csv("nemovitosti.csv")

In [ ]:
df_nemovitosti.head()

<br>

#### Odstranit nepotřebný sloupeček

In [ ]:
df_nemovitosti.columns

In [ ]:
df_bez_bezejmen = df_nemovitosti.drop("Unnamed: 0", axis=1)

In [ ]:
df_bez_bezejmen.head()

In [ ]:
df_bez_bezejmen.loc[:, "price"].max()

In [ ]:
df_bez_bezejmen.loc[:, "price"].min()

In [ ]:
df_bez_bezejmen.loc[:, "area"].sum()

In [ ]:
df_bez_bezejmen.loc[:, "price"].mean()

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [ ]:
from pandas import DataFrame

In [ ]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'číselná hodnota': range(6)},
)

In [ ]:
df_pokus_s_cisly

#### Klasické seskupení podle sloupečku

In [ ]:
df_pokus_s_cisly.groupby("klíč")

In [ ]:
type(df_pokus_s_cisly.groupby("klíč"))

Metoda standardně vrací `DataFrameGroupBy`.

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [ ]:
df_pokus_s_cisly.groupby("klíč").sum()

Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [ ]:
df_pokus_s_cisly

In [ ]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"]

In [ ]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"].median()

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `median`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [ ]:
for (klic, hodnota) in df_pokus_s_cisly.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota.shape}")

<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Agregate

In [ ]:
import numpy

In [ ]:
rng = numpy.random.RandomState(0)

In [ ]:
from pandas import DataFrame

In [ ]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'data_1': range(6),
        'data_2': rng.randint(0, 10, 6)
    }
)

In [ ]:
df_pokus_s_cisly

Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [ ]:
df_pokus_s_cisly.groupby("klíč").aggregate("sum")

In [ ]:
df_pokus_s_cisly.groupby("klíč").aggregate(numpy.median)

In [ ]:
df_pokus_s_cisly.groupby("klíč").aggregate(["min", numpy.median, max])

In [ ]:
df_pokus_s_cisly

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
df_ovoce = DataFrame(ovoce_data)

In [ ]:
df_ovoce.head(6)

In [ ]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': sum})

In [ ]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': 'sum', 'cena': 'mean'})

In [ ]:
vystup

1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Filtrování

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [ ]:
df_ovoce.head(6)

In [ ]:
def vyber_pouze_pobocky_s_prodejem_nad_limit(data, limit: int = 24) -> bool:
    return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_pouze_pobocky_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [ ]:
vysledek = df_ovoce.groupby('pobocka').filter(vyber_pouze_pobocky_s_prodejem_nad_limit)

In [ ]:
vysledek

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Transformace

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [ ]:
df_ovoce

In [ ]:
def vrat_procenta_z_celkoveho_prodeje(udaj) -> float:
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `procento_z_celkoveho_prodeje`:

In [ ]:
df_ovoce['procento_z_celkoveho_prodeje'] = df_ovoce.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [ ]:
df_ovoce

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda `apply`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [ ]:
df_ovoce.head(6)

In [ ]:
def vypocitej_vydelek_za_artikl(seskupene):
    seskupene['vydelek_pobocky'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [ ]:
vystup = df_ovoce.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

In [ ]:
vystup

In [ ]:
df_ovoce.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

In [ ]:
vystup = df_ovoce.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [ ]:
vystup.head(6)

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [ ]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [ ]:
df_rozdily

#### `transform` vrací výsledky ve stejném tvaru

In [ ]:
def vypocitej_sumu(data):
    return data.sum()

In [ ]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_apply

In [ ]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_trans

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

In [ ]:
df_rozdily

In [ ]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [ ]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

In [ ]:
df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [ ]:
from pandas import DataFrame

df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

In [ ]:
df_prodej_hardware.head()

In [ ]:
#Úloha 1
cilove_prodejny = df_prodej_hardware.groupby('prodejna_id') \
                .filter(lambda prodejna: prodejna.pocet_prodanych_ks.sum() >= 30)

In [ ]:
def func_1(seskupene_data, limit: int = 30) -> bool:
    return seskupene_data["pocet_prodanych_ks"] >= limit

In [ ]:
prodeje_prodejen = cilove_prodejny.groupby('prodejna_id') \
                .aggregate({'pocet_prodanych_ks': sum})

In [ ]:
df_prodej_hardware["prodejna_id"].unique()

In [ ]:
#Úloha 2
prijmy_prodejen = pd.DataFrame(df_prodej_hardware.groupby('prodejna_id')['cena_predmetu'] \
                .apply(lambda prodejna: prodejna.sum())) \
                .rename(columns = { 'cena_predmetu': 'prijem_z_prodeje' } ) \
                .round( 2 )

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    def vyber_prodej_vetsi_nez_limit(data, limit: int = 30):
        return data['pocet_prodanych_ks'].sum() > limit
        
    def vypocitej_celkovy_vydelek_prodejny(skupina):
        return (skupina['pocet_prodanych_ks'] * skupina['cena_predmetu']).sum()
    
    df_filtr_hardware = df_prodej_hardware.groupby('prodejna_id') \
                            .filter(vyber_prodej_vetsi_nez_limit)
    
    
    celkovy_vydelek = df_filtr_hardware.groupby('prodejna_id') \
                            .apply(vypocitej_celkovy_vydelek_prodejny)
    ```
</details>

## Pivot tabulky

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [ ]:
!pip install seaborn

In [ ]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [ ]:
df_titanic = seaborn.load_dataset('titanic')

In [ ]:
df_titanic.head()

<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [ ]:
df_titanic["sex"].unique()

In [ ]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [ ]:
df_titanic.groupby('sex')['survived'].mean()

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [ ]:
df_titanic.groupby('sex')['survived'].mean().round(2)

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [ ]:
df_titanic["class"].unique()

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

In [ ]:
type(df_titanic.groupby(['sex', 'class'])[['survived']])

In [ ]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

In [ ]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [ ]:
df_titanic.head()

In [ ]:
df_titanic['age'].describe()

In [ ]:
from pandas import cut

In [ ]:
age = cut(df_titanic['age'], [1, 25, 80])

In [ ]:
age  # < xxx , xxx )

In [ ]:
df_titanic.pivot_table('survived', index=['sex', age], columns='class').round(2)

Pomocí funkce `cut` doplníš tabulku na **Multiindex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

**🧠 CVIČENÍ 🧠, procvič si pivot tabulky**

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
# pivot_table()

In [ ]:
prodane_kusy = DataFrame(ovoce_data) \
            .pivot_table(
                index='pobocka',
                columns ='ovoce',
                values='prodane_mnozstvi'
)

In [ ]:
prodane_kusy

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
    ```
</details>

<br>

## Časové řady

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: Časové řady umožňují analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny. To je klíčové v oblastech jako finance, ekonomika, prodej, počasí a mnoho dalších.

* **Efektivní manipulace s časem**: Pandas poskytuje nástroje pro efektivní práci s časovými řadami, což zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny. Díky integrovaným funkcím můžete snadno pracovat s daty různých časových frekvencí a konvertovat mezi nimi.

* **Časově závislé analýzy**: Časové řady umožňují provádět časově závislé analýzy, jako je detekce změn v trendech, identifikace sezónních vlivů nebo identifikace časových závislostí mezi proměnnými.

* **Flexibilní indexace**: Pandas podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat na základě časových značek nebo rozsahů.

* **Kompatibilita s dalšími knihovnami**: Pandas je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je statsmodels nebo scikit-learn. To usnadňuje integraci a rozšíření vašich analýz s pokročilejšími metodami a algoritmy.
    
<br>

Jde například o údaje typu:
* *timestampy*, údaj odkazující na konkrétní časový okamžik (např. `4. července 2015 v 7:00 hod.`),
* *časové intervaly*, tedy období odkazují na délku času mezi konkrétním začátkem a koncem (např. intervaly ze dne na den),
* *time delta* objekty, tedy přesné délky času (např. 22,22 sekundy).

#### Data a čas v Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [ ]:
from datetime import datetime

In [ ]:
datetime(year=2023, month=4, day=5)

<br>

Nebo knihovna `dateutil` pro parsování datových typů z různých stringových zadání:

In [ ]:
from dateutil import parser

In [ ]:
date = parser.parse("5th of april, 2023")

In [ ]:
date

Kde pomocí metody `strftime` můžeš vypsat den:

In [ ]:
date.strftime("%A")

#### Data a čas v numpy

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [ ]:
from numpy import array, arange

In [ ]:
date = array('2023-04-05', dtype=np.datetime64)

In [ ]:
date

<br>

Pokud potřebuješ pole následujících 7 dní:

In [ ]:
date + arange(7)

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

#### Data a čas v pandách

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [ ]:
from pandas import to_datetime

In [ ]:
date = to_datetime("5th of April, 2023")

In [ ]:
date

In [ ]:
date.strftime("%A")

<br>

### Časové řady

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

#### Indexovní časem
`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy.

Výhody práce s `DatetimeIndex` objektem:
* Časové zóny,
* frekvence,
* časově závislé selekce,
* atributy časových značek,
* operace s časem.

In [ ]:
from pandas import DatetimeIndex, Series

In [ ]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [ ]:
indexy = DatetimeIndex(datumy)

In [ ]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [ ]:
df_hodnoty = Series(hodnoty, index=indexy)

In [ ]:
df_hodnoty

In [ ]:
type(indexy)

In [ ]:
df_hodnoty["2020": "2022"]

<br>

### Základní objekty

---

Mezi základní objekty pro práci s časem patří:
* `Timestamp` typ (související struktura Indexu `DatetimeIndex`,
* `Period` typ (.. `PeriodIndex`),
* `Timedelta` typ (.. `TimedeltaIndex`).

#### Timestamp & DatetimeIndex

Nejčastější datové typy, které lze vyvolat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

In [ ]:
from pandas import to_datetime

#### Práce s jedním datumem

In [ ]:
datum = to_datetime("05/04/2023")

In [ ]:
datum

In [ ]:
type(datum)

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

Pokud se parser ztratí nebo tvoje zadání neodpovídá jeho vyhotovení:

In [ ]:
pd.to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

<br>

#### Práce s několika datumy

In [ ]:
datumy = to_datetime([
    datetime(2023, 4, 5), "5th of April 2023", "2023-Apr-5", "05-04-2023", "20230405"]
)

In [ ]:
datumy

In [ ]:
type(datumy)

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [ ]:
datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

#### Denní data

In [ ]:
datumy.to_period("D")

#### Měsíční data

In [ ]:
datumy.to_period("M")

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
    'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
df_period = df.to_period(freq='Y')

<br>

### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [ ]:
specialni_datumy = to_datetime([
    datetime(2022, 4, 5), "5th of April 2021", "2020-Apr-5", "05-04-2019", "20180405"]
)

In [ ]:
datumy - specialni_datumy

<br>

### Funkce `date_range`

---

Aby bylo zadání řady (sekvence) dat pohodlnější, vyzkoušej funkci `date_range`.

Obdobně potom pracují související funkce:
* `date_range`, timestampy,
* `period_range`, periody,
* `timedelta_range`, pro delty.

In [ ]:
from pandas import date_range

<br>

#### Počet period

In [ ]:
datumy_ind = date_range("01-01-1992", periods=8)

In [ ]:
datumy_ind

<br>

#### Frekvence

In [ ]:
datumy_mesicne_ind = date_range("01-01-1992", periods=8, freq="M")

In [ ]:
datumy_mesicne_ind

<br>

#### Hodinové periody

In [ ]:
from pandas import timedelta_range

In [ ]:
hodinove_ind = timedelta_range(0, periods=12, freq="H")

In [ ]:
hodinove_ind

<br>

### Frekvence

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovníḧo měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

In [ ]:
timedelta_range(0, periods=5, freq="1H15T")

### Resampling

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [ ]:
import pandas as pd

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
monthly_data = df.resample('M').sum()

In [ ]:
print(monthly_data)

<br>

### Zpřesňování

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='MS'),
    'sales': [100, 120, 150]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
daily_data = df.asfreq('D', method='ffill')

In [ ]:
daily_data.head(20)

Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [ ]:
import pandas as pd
import numpy as np

denni_datumy = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = np.random.randint(10, 100, size=(len(date_rng),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = pd.DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

In [ ]:
df_prodeje

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    monthly_sales = df.resample('M').sum()
    print("Celkový prodej za každý měsíc:")
    print(monthly_sales)

    df_prodeje['weekday'] = df_prodeje.index.weekday
    average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
    print("\nPrůměrný prodej za každý den v týdnu:")
    print(average_weekday_sales)

    last_week_sales = df.loc['2022-12-25':, 'sales']
    print("\nDenní prodej za poslední týden:")
    print(last_week_sales)
    ```
</details>

---